In [53]:
# import key libraries
from PIL import Image, ImageDraw
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from scipy import misc
import scipy.ndimage

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [148]:
def collect_mp4_frames(link):
    # create video capture instance
    vidcap = cv2.VideoCapture(link)
    success,image = vidcap.read()
    frames = [image]
    # iterate over video frames, save each in a list
    while success:
        success,image = vidcap.read()
        frames.append(image)
    return frames

def create_predictor():
    # create Detectron2 config and Default Predictor to run image inference
    cfg = get_cfg()
    # add project-specific config here if not running a model in Detectron2's core library
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
    cfg.MODEL.DEVICE = 'cpu'
    # find a model from detectron2's model zoo.
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    # build predictor
    predictor = DefaultPredictor(cfg)
    return predictor

def blur_people(img, predictor):
    # image in tensor form
    im = np.array(img)
    # image dimensions
    height, width, channels = im.shape
    # run inference on the image
    outputs = predictor(im)
    # get indices of predicted instances that are labelled as people
    person_idx = (outputs["instances"].pred_classes==0)
    # count of 'people' instances
    person_count = np.sum(np.array(person_idx))
    # get image masks corresponding to each identified person
    person_masks_tensor = outputs["instances"].pred_masks[person_idx,:,:]
    # create PIL image masks
    person_masks = Image.new('L', im.shape[:-1], 0)
    draw = ImageDraw.Draw(person_masks)    
    # create mask and add peoples' shape to it
    sharp_mask = np.zeros((height, width, channels))
    for i in range(person_count):
        sharp_mask[person_masks_tensor[i,:,:]==1]=1
    # filtered image
    result = (np.multiply(filter_image(img),sharp_mask)+img).astype(dtype=np.uint8)            
    return result

def filter_image(sharp_image):
    # does what it says on the tin
    blurred_image = scipy.ndimage.filters.sobel(sharp_image, mode='constant')
    blurred_image = scipy.ndimage.filters.prewitt(blurred_image, mode='reflect')
    return blurred_image

In [149]:
link = r'/Users/julienraffaud/Downloads/IMG_0240.mp4'

In [150]:
frames = collect_mp4_frames(link)

In [151]:
# use default detectron2 predictor
predictor = create_predictor()

In [172]:
blurred_frames = []
for i in range(len(frames)):
    print ( '%1.2f%% blurred' %(i/len(frames)) )
    blurred_frames.append(blur_people(frames[i],predictor))

0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.0% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.1% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred
0.2% blurred

ValueError: not enough values to unpack (expected 3, got 0)

In [220]:
for i in range(len(blurred_frames)):
    Image.fromarray(blurred_frames[i]).save(r'/Users/julienraffaud/Desktop/Detection/Blurred_pics/blurred-' + str(i)+'.png')

IndexError: list index out of range

In [232]:
import cv2
import os

image_folder = r'/Users/julienraffaud/Desktop/Detection/Blurred_pics/'
video_name = 'blurred_video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]

frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = (1920,1080,3)

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter(video_name, 0, 30, (width,height))

for i in range(331):
    img_name =  'blurred-%d.png' % i
    video.write(cv2.imread(os.path.join(image_folder, img_name)))

# for image in images:
#     video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()